In [89]:
import pandas as pd
import numpy as np
import vectorbt as vbt
from datetime import datetime
import os 
from numba import njit
from vectorbt.portfolio.enums import Direction, SizeType
import talib
import requests
import time



In [90]:


def fetch_kucoin_candles_chunk(symbol, market_type="spot", timeframe="1min", start_time=None, end_time=None):
    time.sleep(0.2)  # Rate limiting
    base_url = "https://api.kucoin.com" if market_type.lower() == "spot" else "https://api-futures.kucoin.com"
    url = base_url + "/api/v1/market/candles"
    params = {"type": timeframe, "symbol": symbol.upper()}
    if start_time:
        params["startAt"] = int(time.mktime(time.strptime(start_time, "%Y-%m-%d %H:%M:%S")))
    if end_time:
        params["endAt"] = int(time.mktime(time.strptime(end_time, "%Y-%m-%d %H:%M:%S")))
    try:
        resp = requests.get(url, params=params, timeout=10)
        resp.raise_for_status()
        data = resp.json()
        if data.get("code") == "200000":
            return data["data"]
        else:
            raise Exception(f"KuCoin API error: {data}")
    except requests.exceptions.RequestException as e:
        raise Exception(f"Request failed: {str(e)}")

def fetch_all_kucoin_candles(symbol, market_type="spot", timeframe="1min", start_time=None, end_time=None):
    chunks = []
    current_end = end_time
    start_timestamp = int(time.mktime(time.strptime(start_time, "%Y-%m-%d %H:%M:%S")))
    
    while True:
        try:
            chunk = fetch_kucoin_candles_chunk(symbol, market_type, timeframe, start_time, current_end)
            if not chunk:
                print("No more data available")
                break
            earliest_ts = int(chunk[-1][0])
            print(f"Fetched {len(chunk)} candles from {datetime.fromtimestamp(earliest_ts)}")
            chunks.extend(chunk)
            if earliest_ts <= start_timestamp:
                print("Reached start time")
                break
            current_end = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime(earliest_ts - 60))
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break
    
    if not chunks:
        return []
    
    chunks.sort(key=lambda x: x[0])  # Sort by timestamp
    result = [candle for candle in chunks if start_timestamp <= int(candle[0]) <= int(time.mktime(time.strptime(end_time, "%Y-%m-%d %H:%M:%S")))]
    print(f"Total candles fetched: {len(result)}")
    return result


symbol = "BTC-USDT"
timeframe = "4hour"
market_type = "spot"
start_time = "2019-07-01 00:00:00"
end_time = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")
# end_time = "2021-06-10 00:00:00"

data = fetch_all_kucoin_candles(symbol, market_type, timeframe, start_time, end_time)
print(f"Final data length: {len(data)}")
data

Fetched 1500 candles from 2024-04-21 13:00:00
Fetched 1500 candles from 2023-08-15 13:00:00
Fetched 1500 candles from 2022-12-08 12:00:00
Fetched 1500 candles from 2022-04-02 13:00:00
Fetched 1500 candles from 2021-07-26 13:00:00
Fetched 1500 candles from 2020-11-18 12:00:00
Fetched 1500 candles from 2020-03-13 12:00:00
Fetched 1500 candles from 2019-07-07 13:00:00
Fetched 39 candles from 2019-07-01 01:00:00
No more data available
Total candles fetched: 12039
Final data length: 12039


[['1561939200',
  '10831.1',
  '11133.8',
  '11251.3',
  '10714.8',
  '283.35380137',
  '3125663.250287782'],
 ['1561953600',
  '11137.1',
  '10984.6',
  '11202.7',
  '10884.1',
  '209.12486824',
  '2313115.732305274'],
 ['1561968000',
  '10982.6',
  '11019.5',
  '11191',
  '10913',
  '210.46833994',
  '2325462.771344483'],
 ['1561982400',
  '11017.5',
  '10385.5',
  '11036',
  '10045.8',
  '410.09598803',
  '4309549.954339466'],
 ['1561996800',
  '10386.9',
  '10255.2',
  '10478',
  '10042.2',
  '296.09692178',
  '3044324.531518726'],
 ['1562011200',
  '10261.2',
  '10630.3',
  '10692',
  '10258.7',
  '271.4723058',
  '2854762.042718658'],
 ['1562025600',
  '10619.5',
  '10340.6',
  '10692',
  '10215.5',
  '210.83038684',
  '2192012.65805174'],
 ['1562040000',
  '10336.6',
  '9870.4',
  '10381.5',
  '9777.7',
  '311.69706875',
  '3089744.397350774'],
 ['1562054400',
  '9860.9',
  '10016',
  '10346',
  '9750',
  '269.96753221',
  '2713621.71105505'],
 ['1562068800',
  '9999.4',
  '1062

In [91]:

# Assuming 'data' is the list of candlestick data
df = pd.DataFrame(data)
df[['timestamp', 'open', 'close', 'high', 'low']] = df[[0, 1, 2, 3, 4]]
df.drop([0, 1, 2, 3, 4, 5, 6], axis=1, inplace=True)

# Explicitly cast 'timestamp' to numeric type
df['timestamp'] = pd.to_numeric(df['timestamp'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s', utc=True)

df.set_index('timestamp', inplace=True)
df.sort_index(inplace=True)
df[['open', 'close', 'high', 'low']] = df[['open', 'close', 'high', 'low']].astype(float)
df.info()
df

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 12039 entries, 2019-07-01 00:00:00+00:00 to 2024-12-27 08:00:00+00:00
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    12039 non-null  float64
 1   close   12039 non-null  float64
 2   high    12039 non-null  float64
 3   low     12039 non-null  float64
dtypes: float64(4)
memory usage: 470.3 KB


,open,close,high,low
timestamp,,,,
2019-07-01 00:00:00+00:00,10831.1,11133.8,11251.3,10714.8
2019-07-01 04:00:00+00:00,11137.1,10984.6,11202.7,10884.1
2019-07-01 08:00:00+00:00,10982.6,11019.5,11191.0,10913.0
2019-07-01 12:00:00+00:00,11017.5,10385.5,11036.0,10045.8
2019-07-01 16:00:00+00:00,10386.9,10255.2,10478.0,10042.2
...,...,...,...,...
2024-12-26 16:00:00+00:00,96043.3,96096.6,96638.8,95443.3
2024-12-26 20:00:00+00:00,96096.6,95785.8,96116.8,95311.4
2024-12-27 00:00:00+00:00,95785.9,96423.4,96444.5,95402.2


## creating indicator to set signals long ONLY for simplicity

In [92]:
# BBANDS = vbt.IndicatorFactory.from_talib('BBANDS')
# bbands = BBANDS.run(df['close'], timeperiod=20)
# print(type(bbands))


# bbands_vbt = vbt.BBANDS.run(df['close'])
# bbands_upper = bbands_vbt.upper
# print(type(bbands_vbt))
# print(type(bbands_upper))
# print(type(bbands_upper.to_numpy()))
# print(bbands_upper.to_numpy().shape)

# # upper_band, middle_band, lower_band = talib.BBANDS(df['close'], timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
# # upper_band

In [93]:
skip_cell = False

if not skip_cell:
    print("This cell is executed.")

    def ma_cross_rsi_long_only(close_price,open_price, ma1_fast, ma2_slow, rsi_buy,rsi_window,bb_window,ewm_use,str_dev):
        SMA = vbt.IndicatorFactory.from_talib('SMA')
        ma1 = SMA.run(close_price, timeperiod=ma1_fast).real.to_numpy()
        ma2 = SMA.run(close_price, timeperiod=ma2_slow).real.to_numpy()

        RSI = vbt.IndicatorFactory.from_talib('RSI')
        rsi = RSI.run(close_price, timeperiod=rsi_window).real.to_numpy()

        bbands = vbt.BBANDS.run(df['close'], bb_window,ewm=ewm_use,alpha=str_dev)
        upper_band = bbands.upper.to_numpy().reshape(-1,1)
        # print(upper_band.shape)
        # print(rsi.shape)
        # print(ma1.shape)
        # print(ma2.shape)
        bb_buy = (close_price > upper_band) & (open_price < upper_band) 

        rsi_buy_signal = rsi > rsi_buy
        signal_buy = (close_price > ma1) & rsi_buy_signal & (ma1 > ma2) & bb_buy
        caution_long = (close_price < ma1) & (ma1 > ma2)
        # Check if ma1 and ma2 are too close to each other (percentage difference)
        percentage_diff = np.abs(ma1 - ma2) / ma2
        undecided = percentage_diff < 0.02
        # Define conditions and corresponding values for signals
        conditions = [
            signal_buy & ~undecided,
            caution_long & ~undecided,
            undecided
        ]

        values = [
            100,  # Strong buy signal (close price above short trend MA and RSI above the buy threshold)
            50,   # Caution long
            0     # Undecided
        ]

        # Use np.select to apply the conditions and assign the corresponding values
        signal = np.select(conditions, values, default=0)  # 0: No signal (default value when none of the conditions are met)

        return signal

    # Create the indicator factory
    ma_cross_slow_long_only = vbt.IndicatorFactory(
        class_name='MA_Cross_rsi',
        short_name='ma_rsi',
        input_names=['Close','Open'],
        param_names=['ma1_fast', 'ma2_slow', 'rsi_buy', 'rsi_window','bb_window','ewm_use','str_dev'],
        output_names=['ma_rsi']
    ).from_apply_func(ma_cross_rsi_long_only)

    # # # Define window sizes and RSI thresholds using np.arange
    ma_trend_fast = np.arange(20, 61, 10)  # Range from 20 to 50 with step 5
    ma_trend_slow = np.arange(250, 551, 50)  # Range from 50 to 100 with step 10
    rsi_buy = np.arange(50, 61, 5)  # Range from 60 to 80 with step 5
    # rsi_window = np.arange(14, 25, 2)  # Range from 10 to 20 with step 2
    bb_window = np.arange(10, 26, 5)  # Range from 10 to 30 with step 5
    # ewm_use = [False, True]
    # str_dev = np.arange(1, 3.1, 0.5)  
    
    
    # ma_trend_fast = 50
    # ma_trend_slow = 400
    # rsi_buy = 70
    rsi_window = 14
    # bb_window = 20
    ewm_use = False
    str_dev = [1.5,1.7,2]


    # Run the indicator
    signal = ma_cross_slow_long_only.run(
        df['close'],df['open'],
        ma1_fast=ma_trend_fast,
        ma2_slow=ma_trend_slow,
        rsi_buy=rsi_buy,
        rsi_window=rsi_window,
        bb_window =bb_window,
        ewm_use= ewm_use,
        str_dev = str_dev,
        param_product=True  
    )

    signal.ma_rsi

else:
    print("This cell is skipped.")

This cell is executed.


In [94]:
signal.ma_rsi

ma_rsi_ma1_fast              20                                            \
ma_rsi_ma2_slow             250                                             
ma_rsi_rsi_buy               50                                             
ma_rsi_rsi_window            14                                             
ma_rsi_bb_window             10            15            20            25   
ma_rsi_ewm_use            False         False         False         False   
ma_rsi_str_dev              1.5 1.7 2.0   1.5 1.7 2.0   1.5 1.7 2.0   1.5   
timestamp                                                                   
2019-07-01 00:00:00+00:00     0   0   0     0   0   0     0   0   0     0   
2019-07-01 04:00:00+00:00     0   0   0     0   0   0     0   0   0     0   
2019-07-01 08:00:00+00:00     0   0   0     0   0   0     0   0   0     0   
2019-07-01 12:00:00+00:00     0   0   0     0   0   0     0   0   0     0   
2019-07-01 16:00:00+00:00     0   0   0     0   0   0     0   0   0     0   
...                         ...  ..  ..   ...  ..  ..   ...  ..  ..   ...   
2024-12-26 16:00:00+00:00     0   0   0     0   0   0     0   0   0     0   
2024-12-26 20:00:00+00:00     0   0   0     0   0   0     0   0   0     0   
2024-12-27 00:00:00+00:00     0   0   0     0   0   0     0   0   0     0   
2024-12-27 04:00:00+00:00     0   0   0     0   0   0     0   0   0     0   
2024-12-27 08:00:00+00:00     0   0   0     0   0   0     0   0   0     0   

ma_rsi_ma1_fast            ...    60                                            
ma_rsi_ma2_slow            ...   550                                            
ma_rsi_rsi_buy             ...    60                                            
ma_rsi_rsi_window          ...    14                                            
ma_rsi_bb_window           ...    10    15            20            25          
ma_rsi_ewm_use             ... False False         False         False          
ma_rsi_str_dev             ...   2.0   1.5 1.7 2.0   1.5 1.7 2.0   1.5 1.7 2.0  
timestamp                  ...                                                  
2019-07-01 00:00:00+00:00  ...     0     0   0   0     0   0   0     0   0   0  
2019-07-01 04:00:00+00:00  ...     0     0   0   0     0   0   0     0   0   0  
2019-07-01 08:00:00+00:00  ...     0     0   0   0     0   0   0     0   0   0  
2019-07-01 12:00:00+00:00  ...     0     0   0   0     0   0   0     0   0   0  
2019-07-01 16:00:00+00:00  ...     0     0   0   0     0   0   0     0   0   0  
...                        ...   ...   ...  ..  ..   ...  ..  ..   ...  ..  ..  
2024-12-26 16:00:00+00:00  ...    50    50  50  50    50  50  50    50  50  50  
2024-12-26 20:00:00+00:00  ...    50    50  50  50    50  50  50    50  50  50  
2024-12-27 00:00:00+00:00  ...    50    50  50  50    50  50  50    50  50  50  
2024-12-27 04:00:00+00:00  ...    50    50  50  50    50  50  50    50  50  50  
2024-12-27 08:00:00+00:00  ...    50    50  50  50    50  50  50    50  50  50  

[12039 rows x 1260 columns]

In [95]:
def repeat_series_horizontally(series, target_shape):
    data = series.values.reshape(-1, 1)  # Convert to column vector
    return np.repeat(data, target_shape[1], axis=1)

nedded_copies = len(signal.ma_rsi.columns)
print('needed columns: ',nedded_copies)
print('needed rows: ',len(signal.ma_rsi))
print('shape:', signal.ma_rsi.shape)
print()

# copying the close price to the shape of the indicator
print('original shape:\n\n', df['close'])
dublicate_close = repeat_series_horizontally(df['close'], signal.ma_rsi.shape)
df_dublicate_close  = pd.DataFrame(dublicate_close,df.index)
df_dublicate_close 



needed columns:  1260
needed rows:  12039
shape: (12039, 1260)

original shape:

 timestamp
2019-07-01 00:00:00+00:00    11133.8
2019-07-01 04:00:00+00:00    10984.6
2019-07-01 08:00:00+00:00    11019.5
2019-07-01 12:00:00+00:00    10385.5
2019-07-01 16:00:00+00:00    10255.2
                              ...   
2024-12-26 16:00:00+00:00    96096.6
2024-12-26 20:00:00+00:00    95785.8
2024-12-27 00:00:00+00:00    96423.4
2024-12-27 04:00:00+00:00    95106.4
2024-12-27 08:00:00+00:00    96464.6
Name: close, Length: 12039, dtype: float64


,0,1,2,3,4,5,6,7,8,9,...,1250,1251,1252,1253,1254,1255,1256,1257,1258,1259
timestamp,,,,,,,,,,,,,,,,,,,,,
2019-07-01 00:00:00+00:00,11133.8,11133.8,11133.8,11133.8,11133.8,11133.8,11133.8,11133.8,11133.8,11133.8,...,11133.8,11133.8,11133.8,11133.8,11133.8,11133.8,11133.8,11133.8,11133.8,11133.8
2019-07-01 04:00:00+00:00,10984.6,10984.6,10984.6,10984.6,10984.6,10984.6,10984.6,10984.6,10984.6,10984.6,...,10984.6,10984.6,10984.6,10984.6,10984.6,10984.6,10984.6,10984.6,10984.6,10984.6
2019-07-01 08:00:00+00:00,11019.5,11019.5,11019.5,11019.5,11019.5,11019.5,11019.5,11019.5,11019.5,11019.5,...,11019.5,11019.5,11019.5,11019.5,11019.5,11019.5,11019.5,11019.5,11019.5,11019.5
2019-07-01 12:00:00+00:00,10385.5,10385.5,10385.5,10385.5,10385.5,10385.5,10385.5,10385.5,10385.5,10385.5,...,10385.5,10385.5,10385.5,10385.5,10385.5,10385.5,10385.5,10385.5,10385.5,10385.5
2019-07-01 16:00:00+00:00,10255.2,10255.2,10255.2,10255.2,10255.2,10255.2,10255.2,10255.2,10255.2,10255.2,...,10255.2,10255.2,10255.2,10255.2,10255.2,10255.2,10255.2,10255.2,10255.2,10255.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-26 16:00:00+00:00,96096.6,96096.6,96096.6,96096.6,96096.6,96096.6,96096.6,96096.6,96096.6,96096.6,...,96096.6,96096.6,96096.6,96096.6,96096.6,96096.6,96096.6,96096.6,96096.6,96096.6
2024-12-26 20:00:00+00:00,95785.8,95785.8,95785.8,95785.8,95785.8,95785.8,95785.8,95785.8,95785.8,95785.8,...,95785.8,95785.8,95785.8,95785.8,95785.8,95785.8,95785.8,95785.8,95785.8,95785.8
2024-12-27 00:00:00+00:00,96423.4,96423.4,96423.4,96423.4,96423.4,96423.4,96423.4,96423.4,96423.4,96423.4,...,96423.4,96423.4,96423.4,96423.4,96423.4,96423.4,96423.4,96423.4,96423.4,96423.4


## compare the shape of the copied closing prices 

In [96]:
df_custom_indiator = signal.ma_rsi

if df_custom_indiator.shape == df_dublicate_close.shape:
    df_renamed_close = df_dublicate_close
    df_renamed_close.columns = df_custom_indiator.columns
    print("Columns have been renamed ")
else:
    print("The DataFrames do not have the same shape.")

# Display the resulting DataFrame
df_renamed_close.head()
df_renamed_close

Columns have been renamed 


ma_rsi_ma1_fast                 20                                      \
ma_rsi_ma2_slow                250                                       
ma_rsi_rsi_buy                  50                                       
ma_rsi_rsi_window               14                                       
ma_rsi_bb_window                10                         15            
ma_rsi_ewm_use               False                      False            
ma_rsi_str_dev                 1.5      1.7      2.0      1.5      1.7   
timestamp                                                                
2019-07-01 00:00:00+00:00  11133.8  11133.8  11133.8  11133.8  11133.8   
2019-07-01 04:00:00+00:00  10984.6  10984.6  10984.6  10984.6  10984.6   
2019-07-01 08:00:00+00:00  11019.5  11019.5  11019.5  11019.5  11019.5   
2019-07-01 12:00:00+00:00  10385.5  10385.5  10385.5  10385.5  10385.5   
2019-07-01 16:00:00+00:00  10255.2  10255.2  10255.2  10255.2  10255.2   
...                            ...      ...      ...      ...      ...   
2024-12-26 16:00:00+00:00  96096.6  96096.6  96096.6  96096.6  96096.6   
2024-12-26 20:00:00+00:00  95785.8  95785.8  95785.8  95785.8  95785.8   
2024-12-27 00:00:00+00:00  96423.4  96423.4  96423.4  96423.4  96423.4   
2024-12-27 04:00:00+00:00  95106.4  95106.4  95106.4  95106.4  95106.4   
2024-12-27 08:00:00+00:00  96464.6  96464.6  96464.6  96464.6  96464.6   

ma_rsi_ma1_fast                                                         ...  \
ma_rsi_ma2_slow                                                         ...   
ma_rsi_rsi_buy                                                          ...   
ma_rsi_rsi_window                                                       ...   
ma_rsi_bb_window                         20                         25  ...   
ma_rsi_ewm_use                        False                      False  ...   
ma_rsi_str_dev                 2.0      1.5      1.7      2.0      1.5  ...   
timestamp                                                               ...   
2019-07-01 00:00:00+00:00  11133.8  11133.8  11133.8  11133.8  11133.8  ...   
2019-07-01 04:00:00+00:00  10984.6  10984.6  10984.6  10984.6  10984.6  ...   
2019-07-01 08:00:00+00:00  11019.5  11019.5  11019.5  11019.5  11019.5  ...   
2019-07-01 12:00:00+00:00  10385.5  10385.5  10385.5  10385.5  10385.5  ...   
2019-07-01 16:00:00+00:00  10255.2  10255.2  10255.2  10255.2  10255.2  ...   
...                            ...      ...      ...      ...      ...  ...   
2024-12-26 16:00:00+00:00  96096.6  96096.6  96096.6  96096.6  96096.6  ...   
2024-12-26 20:00:00+00:00  95785.8  95785.8  95785.8  95785.8  95785.8  ...   
2024-12-27 00:00:00+00:00  96423.4  96423.4  96423.4  96423.4  96423.4  ...   
2024-12-27 04:00:00+00:00  95106.4  95106.4  95106.4  95106.4  95106.4  ...   
2024-12-27 08:00:00+00:00  96464.6  96464.6  96464.6  96464.6  96464.6  ...   

ma_rsi_ma1_fast                 60                                      \
ma_rsi_ma2_slow                550                                       
ma_rsi_rsi_buy                  60                                       
ma_rsi_rsi_window               14                                       
ma_rsi_bb_window                10       15                         20   
ma_rsi_ewm_use               False    False                      False   
ma_rsi_str_dev                 2.0      1.5      1.7      2.0      1.5   
timestamp                                                                
2019-07-01 00:00:00+00:00  11133.8  11133.8  11133.8  11133.8  11133.8   
2019-07-01 04:00:00+00:00  10984.6  10984.6  10984.6  10984.6  10984.6   
2019-07-01 08:00:00+00:00  11019.5  11019.5  11019.5  11019.5  11019.5   
2019-07-01 12:00:00+00:00  10385.5  10385.5  10385.5  10385.5  10385.5   
2019-07-01 16:00:00+00:00  10255.2  10255.2  10255.2  10255.2  10255.2   
...                            ...      ...      ...      ...      ...   
2024-12-26 16:00:00+00:00  96096.6  96096.6  96096.6  9

In [97]:
# Numba-compiled order function
@njit
def order_func_nb(c, high, low, open_, entries, sl_prices, tp_prices,entry_price):
    close_price = c.close[c.i, c.col]
    # print("INDEX", c.i)
    # print("COL", c.col)
    # print("high", high[c.i])
    # print('open', open_[c.i])
    # print("close", close_price)
    # print('low', low[c.i])
    # print('position size :', c.position_now)
    # print('cash:', c.cash_now)
    # print('entries:', entries[c.i, c.col])
    # print('sl_prices:', sl_prices[c.i])
    # print('tp_prices:', tp_prices[c.i])
    # print('entry_price:', entry_price[c.i])
    # print()

# if in position 
    if (c.position_now > 0):
        # Check if SL is hit
        if low[c.i] <= sl_prices[c.i]:
            value = vbt.portfolio.nb.order_nb(
                size=-np.inf,  # Close position
                price=sl_prices[c.i],
                size_type=SizeType.Amount,
                direction=Direction.LongOnly,
                slippage=0.01)
            # print('sl hit at index',c.i)
            # print('sl order', value)
            return value

        # sl update for long position

        if c.i >= 5:  # Ensure there are enough bars to look back
            lowest_low = np.min(low[c.i-2:c.i])  # Find the lowest low in the last 5 bars
            if (entries[c.i, c.col] == 75):
                # print('sl update before',c.i,sl_prices[c.i])
                update1 = close_price * 0.90
                if update1 > sl_prices[c.i]:
                    sl_prices[:]= update1
                    # print('sl update after at index',c.i,sl_prices[c.i])
            if (entries[c.i, c.col] == 50):  
                update2 = close_price * 0.95
                if update2 > sl_prices[c.i]:
                    # print('sl update before',c.i,sl_prices[c.i])
                    sl_prices[:]= update2       
            if (entries[c.i, c.col] == 0):  
                update3 = close_price * 0.98
                if update3 > sl_prices[c.i]:
                    # print('sl update before',c.i,sl_prices[c.i])
                    sl_prices[:]= update3      



        # if c.position_now > 0:
        #     if (high[c.i-1,c.col] >= tp_prices[c.i])&(tp_prices[c.i] != np.nan):
        #         value = vbt.portfolio.nb.order_nb(
        #             size=-c.position_now/2,  # Close position
        #             price=tp_prices[c.i],
        #             size_type=SizeType.Amount,
        #             direction=Direction.LongOnly)
        #         tp_prices[:] = np.nan
        #         sl_prices[:] = entry_price[c.i]
        #         # print('tp hit at index',c.i)
        #         return value
                


    # if not in position search for position to enter
    elif (c.position_now == 0) & (c.i != 0):
        if entries[c.i, c.col] == 100:
            entry_price[:] = close_price
            tp_prices[:] = close_price * 1.05
            order = vbt.portfolio.nb.order_nb(
                size=0.10,  # Adjusted order size
                price=close_price,  # Current closing price
                size_type=SizeType.Percent,  # Specify size type
                direction=Direction.LongOnly,  # Long-only trading
                fees=0.01,  # No fees
                slippage=0.01,  # No slippage
                allow_partial=False,  # Do not allow partial fills
                raise_reject=True  # Raise an error if the order is rejected
            )
            lowest_low = np.min(low[c.i-5:c.i]) 
            sl_prices[:] = lowest_low * 0.98
            # print('order', order)
            return order

    
    return vbt.portfolio.enums.NoOrder


# open_ =repeat_series_horizontally(df['open'],signal.ma_rsi.shape)
# print(type(open_))
# high = repeat_series_horizontally(df['high'],signal.ma_rsi.shape)
# low = repeat_series_horizontally(df['low'],signal.ma_rsi.shape)
close = df_renamed_close
entries = df_custom_indiator.to_numpy()

open_ = df['open'].to_numpy().flatten()
high = df['high'].to_numpy().flatten()
low = df['low'].to_numpy().flatten()



# Create an array to store SL prices
sl_prices = np.full(close.shape[0], np.nan)  # Use a 1D array
tp_prices = np.full(close.shape[0], np.nan)  # Use a 1D array
entry_price = np.full(close.shape[0], np.nan)  # Use a 1D array
# Create portfolio
pf = vbt.Portfolio.from_order_func(
    close,           # Price DataFrame
    order_func_nb,
    high,
    low,
    open_,
    entries,    # Order function
    sl_prices,
    tp_prices,
    entry_price,  # Pass the SL prices array
    init_cash=500  # Initial cash balance
)

# Display some portfolio performance metrics
print("Total Return:", pf.total_return())
print("\nOrder Records:")

pf.orders.records_readable

# Optional: Plot equity curve
# pf.plot().show()



Total Return: ma_rsi_ma1_fast  ma_rsi_ma2_slow  ma_rsi_rsi_buy  ma_rsi_rsi_window  ma_rsi_bb_window  ma_rsi_ewm_use  ma_rsi_str_dev
20               250              50              14                 10                False           1.5              -0.395658
                                                                                                       1.7              -0.371969
                                                                                                       2.0              -0.330228
                                                                     15                False           1.5              -0.393789
                                                                                                       1.7              -0.358772
                                                                                                                           ...   
60               550              60              14                 20                F

,Order Id,Column,Timestamp,Size,Price,Fees,Side
0,0,"(20, 250, 50, 14, 10, False, 1.5)",2019-11-02 12:00:00+00:00,0.005259,9413.50300,0.495050,Buy
1,1,"(20, 250, 50, 14, 10, False, 1.5)",2019-11-03 08:00:00+00:00,0.005259,9048.37626,0.000000,Sell
2,2,"(20, 250, 50, 14, 10, False, 1.5)",2019-11-04 20:00:00+00:00,0.005192,9488.84900,0.492658,Buy
3,3,"(20, 250, 50, 14, 10, False, 1.5)",2019-11-05 04:00:00+00:00,0.005192,9120.65616,0.000000,Sell
4,4,"(20, 250, 50, 14, 10, False, 1.5)",2019-11-10 16:00:00+00:00,0.005366,9136.56100,0.490278,Buy
...,...,...,...,...,...,...,...
405265,405265,"(60, 550, 60, 14, 25, False, 2.0)",2024-11-21 12:00:00+00:00,0.000405,95055.92712,0.000000,Sell
405266,405266,"(60, 550, 60, 14, 25, False, 2.0)",2024-12-04 20:00:00+00:00,0.000383,99604.48300,0.381779,Buy
405267,405267,"(60, 550, 60, 14, 25, False, 2.0)",2024-12-05 16:00:00+00:00,0.000383,99493.81596,0.000000,Sell
405268,405268,"(60, 550, 60, 14, 25, False, 2.0)",2024-12-15 16:00:00+00:00,0.000366,104280.98500,0.381359,Buy


In [98]:
total_return = pf.total_return()
max_dd = pf.max_drawdown()
return_and_maxdd = pd.concat([total_return, max_dd], axis=1)
return_and_maxdd.sort_values(by='total_return', ascending=False).head(20)

total_return  \
ma_rsi_ma1_fast ma_rsi_ma2_slow ma_rsi_rsi_buy ma_rsi_rsi_window ma_rsi_bb_window ma_rsi_ewm_use ma_rsi_str_dev                 
20              400             60             14                20               False          2.0                -0.144135   
                                50             14                20               False          2.0                -0.151783   
                                55             14                20               False          2.0                -0.154234   
50              250             60             14                20               False          2.0                -0.156393   
20              350             60             14                20               False          2.0                -0.156658   
50              350             60             14                20               False          2.0                -0.157921   
30              400             60             14                20               False          2.0                -0.158046   
60              250             60             14                20               False          2.0                -0.161229   
20              300             60             14                20               False          2.0                -0.161655   
30              300             60             14                20               False          2.0                -0.162487   
40              350             60             14                20               False          2.0                -0.163482   
                400             60             14                20               False          2.0                -0.163601   
30              350             60             14                20               False          2.0                -0.164689   
60              350             60             14                20               False          2.0                -0.166418   
30              400             50             14                20               False          2.0                -0.166537   
                250             60             14                20               False          2.0                -0.167526   
                400             55             14                20               False          2.0                -0.167981   
40              250             60             14                20               False          2.0                -0.168180   
20              350             50             14                20               False          2.0                -0.168519   
                450             60             14                20               False          2.0                -0.169237   

                                                                                                                 max_drawdown  
ma_rsi_ma1_fast ma_rsi_ma2_slow ma_rsi_rsi_buy ma_rsi_rsi_window ma_rsi_bb_window ma_rsi_ewm_use ma_rsi_str_dev                
20              400             60             14                20               False          2.0                -0.159028  
                                50             14                20               False          2.0                -0.167140  
                                55             14                20               False          2.0                -0.168952  
50              250             60             14                20               False          2.0                -0.169290  
20              350             60             14                20               False          2.0                -0.169552  
50              350             60             14                20               False          2.0                -0.170795  
30              400             60             14                20               False          2.0                -0.172697  
60              250             60             14                20               False          2.0                -0.174

In [99]:
pf.get_drawdowns().stats()

/var/folders/33/gyg8_bbj4qn5_tvszhbzjxdm0000gn/T/ipykernel_66819/1963487665.py:1: UserWarning:

Object has multiple columns. Aggregating using <function mean at 0x10bd0a9d0>. Pass column to select a single column/group.



Start                             2019-07-01 00:00:00+00:00
End                               2024-12-27 08:00:00+00:00
Period                                   2006 days 12:00:00
Coverage [%]                                      91.750218
Total Records                                      1.773016
Total Recovered Drawdowns                          0.773016
Total Active Drawdowns                                  1.0
Active Drawdown [%]                               30.810971
Active Duration                1832 days 23:32:57.142857152
Active Recovery [%]                                1.010336
Active Recovery Return [%]                         0.325977
Active Recovery Duration         20 days 08:16:57.142857142
Max Drawdown [%]                                    0.48587
Avg Drawdown [%]                                     0.3512
Max Drawdown Duration            15 days 21:06:42.797202797
Avg Drawdown Duration             9 days 22:20:10.189810189
Max Recovery Return [%]                 

In [100]:

df_sorted = pf.total_return()
df_sorted.vbt.volume(x_level='ma_rsi_ma1_fast', y_level='ma_rsi_ma2_slow', z_level='ma_rsi_rsi_buy')


FigureWidget({
    'data': [{'colorscale': [[0.0, '#0d0887'], [0.1111111111111111, '#46039f'],
                             [0.2222222222222222, '#7201a8'], [0.3333333333333333,
                             '#9c179e'], [0.4444444444444444, '#bd3786'],
                             [0.5555555555555556, '#d8576b'], [0.6666666666666666,
                             '#ed7953'], [0.7777777777777778, '#fb9f3a'],
                             [0.8888888888888888, '#fdca26'], [1.0, '#f0f921']],
              'hovertemplate': ('ma_rsi_ma1_fast: %{x}<br>ma_rs' ... 'value: %{value}<extra></extra>'),
              'opacity': 0.2,
              'surface': {'count': 15},
              'type': 'volume',
              'uid': '0465c9e0-6642-455f-9c3e-9947d310f447',
              'value': array([-0.23567656, -0.23630738, -0.22528531, -0.22447446, -0.22511452,
                              -0.21370636, -0.20519562, -0.20585159, -0.19476925, -0.19298524,
                              -0.19365129, -0.1826814

In [101]:

df_sorted = pf.total_return()
df_sorted.vbt.heatmap(x_level='ma_rsi_ma1_fast', y_level='ma_rsi_ma2_slow', slider_level='ma_rsi_rsi_buy')

FigureWidget({
    'data': [{'colorscale': [[0.0, '#0d0887'], [0.1111111111111111, '#46039f'],
                             [0.2222222222222222, '#7201a8'], [0.3333333333333333,
                             '#9c179e'], [0.4444444444444444, '#bd3786'],
                             [0.5555555555555556, '#d8576b'], [0.6666666666666666,
                             '#ed7953'], [0.7777777777777778, '#fb9f3a'],
                             [0.8888888888888888, '#fdca26'], [1.0, '#f0f921']],
              'hoverongaps': False,
              'hovertemplate': 'ma_rsi_ma1_fast: %{x}<br>ma_rsi_ma2_slow: %{y}<br>value: %{z}<extra></extra>',
              'name': '50',
              'type': 'heatmap',
              'uid': '5826c6df-29b1-44f6-9a39-6dea6703f7d0',
              'visible': True,
              'x': array([20, 30, 40, 50, 60]),
              'y': array([250, 300, 350, 400, 450, 500, 550]),
              'z': array([[-0.23567656, -0.22394967, -0.22581772, -0.21588682, -0.21556139],
      

In [102]:
# pf.plot().show()